In [184]:
#we will need some of these packages at varoius points
import numpy as np
import os
from scipy import ndimage
import tensorflow as tf

"""
This code processes images of handwritten letters and creates a CNN to classify these images
We include every lower case letter as well as select upper case letters
In addition the model will be trained to recognize various types of images that are not letters such as empty images, and images that only have straight lines
"""
    
    
    
    
def model(features,labels,mode):
    #this function defines a custom model
    input_layer = tf.reshape(features["x"], [-1, 100, 100, 1])#100x100
    #perform convolutions with relu at the end of each convolution and some dropout, then pooling
    conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=64,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)
    
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)#50x50
    dropoutp1=tf.layers.dropout(inputs=pool1,rate=0.05,training=mode==tf.estimator.ModeKeys.TRAIN)
    
    conv2 = tf.layers.conv2d(
      inputs=dropoutp1,
      filters=32,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)
  
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)#25x25
    dropoutp2=tf.layers.dropout(inputs=pool2,rate=0.06,training=mode==tf.estimator.ModeKeys.TRAIN)
    
    conv3 = tf.layers.conv2d(
      inputs=dropoutp2,
      filters=24,
      kernel_size=[3,3],
      padding="same",
      activation=tf.nn.relu)
      
    pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[5,5], strides=5)#5x5
    dropoutp3=tf.layers.dropout(inputs=pool3,rate=0.07,training=mode==tf.estimator.ModeKeys.TRAIN)
    
    conv4 = tf.layers.conv2d(
      inputs=dropoutp3,
      filters=16,
      kernel_size=[3,3],
      padding="same",
      activation=tf.nn.relu)
    
    #flatten the images which are 5x5 but is also 16 units deep (because our last convolution layer had 16 filters) to give total length of 3*3*81
    pool4_flat=tf.reshape(conv4, [-1,16*25])
    #define our dense layer
    dense=tf.layers.dense(inputs=pool4_flat,units=256,activation=tf.nn.elu)
    #do one final dropout, this time much more dropout than we saw earlier
    dropout=tf.layers.dropout(inputs=dense,rate=0.4,training=mode==tf.estimator.ModeKeys.TRAIN)
    logits=tf.layers.dense(inputs=dropout,units=3)
    
    predictions={
      "classes":tf.argmax(input=logits,axis=1),
      "probabilities":tf.nn.softmax(logits,name="softmax_tensor")
      }
    
    #perform the operations appropriate for the mode we are in
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode,predictions=predictions)
    
    labels_one_hot=tf.one_hot(indices=tf.cast(labels, tf.int32), depth=3)
    loss = tf.losses.softmax_cross_entropy(onehot_labels=labels_one_hot, logits=logits)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer=tf.train.GradientDescentOptimizer(learning_rate=.01)
        train_op = optimizer.minimize(
          loss=loss,
          global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    if mode == tf.estimator.ModeKeys.EVAL:
        eval_metric_ops = {
          "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)
        
def get_feature_dict(images,labes):
    #simply makes a dictionary mapping input (x's) to correct output (y's)
    return {"x":images,"y":labels}
        
def train(train_data,train_labels,eval_data,eval_labels,model_folder,batch_size,steps):
    #we take a a folder with images to train on
    #a folder to evaluate our model with
    #and a folder to save our model in, if this folder already has a model it will attempt to restore that model
    #Raises an error if the model folder contains a model that does not match the format defined in our model function

    classifier= tf.estimator.Estimator(model_fn = model, model_dir = model_folder)
    #defined using our model function above
    #tensors_to_log = {"probabilities": "softmax_tensor"}
    #logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50)
    #we can see the above log results for every 50 steps during training
    
    #Train the model
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": train_data},
        y=train_labels,
        batch_size=batch_size,
        num_epochs=None,
        shuffle=True)
    classifier.train(
        input_fn=train_input_fn,
        steps=steps)
    #hooks=[logging_hook])
    
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": eval_data},
        y=eval_labels,
        num_epochs=1,
        shuffle=False)
    eval_results = classifier.evaluate(input_fn=eval_input_fn)
    print(eval_results)
    
def predict(image,model_folder):
    #this model takes an image and a folder with a trained model and makes a prediction using the model
    classifier= tf.estimator.Estimator(model_fn = model, model_dir = model_folder)
    tensors_to_log = {"probabilities": "softmax_tensor"}
    logging_hook = tf.train.LoggingTensorHook(
        tensors=tensors_to_log, every_n_iter=100)

    # predict with the model and print results
    pred_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": image},
        shuffle=False)
    pred_results = list(classifier.predict(input_fn=pred_input_fn))
    return [p['classes'] for p in pred_results],[p['probabilities'] for p in pred_results]
    
    


In [171]:
#functions for creating dummy data to practice with
def data(num_samps,w1,w2):
    x=np.random.randn(num_samps,100)
    return x, get_y(x,w1,w2)
    
def get_y(x,w1,w2):
    y=np.exp(w1*x.T).T @ w2
    return y
    
def make_image(sample):
    height=[0]
    for i in sample:
        height.append(int(height[-1]+10*i))
    height=np.array(height)-min(height)
    height=(100*height/max(height)).astype('int')
    rows=max(height)
    image=np.zeros((rows,len(sample)))
    for i in range(len(height)-1):
        top=max(height[i-1],height[i])
        bot=min(height[i-1],height[i])
        image[bot:top,i]=1
    return image